In [ ]:
!pip install numpy==1.23.5
!pip install pandas==1.23.5
!pip install openai==0.28.0
!pip install tts diffusers nest_asyncio langchain python-dotenv
!pip install SpeechRecognition pydub
!pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1+cu116
!pip install accelerate
!pip install gradio
!pip install gdown


ERROR: Could not find a version that satisfies the requirement pandas==1.23.5 (from versions: 0.1, 0.2, 0.3.0, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.5.0, 0.6.0, 0.6.1, 0.7.0, 0.7.1, 0.7.2, 0.7.3, 0.8.0, 0.8.1, 0.9.0, 0.9.1, 0.10.0, 0.10.1, 0.11.0, 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1, 0.15.0, 0.15.1, 0.15.2, 0.16.0, 0.16.1, 0.16.2, 0.17.0, 0.17.1, 0.18.0, 0.18.1, 0.19.0, 0.19.1, 0.19.2, 0.20.0, 0.20.1, 0.20.2, 0.20.3, 0.21.0, 0.21.1, 0.22.0, 0.23.0, 0.23.1, 0.23.2, 0.23.3, 0.23.4, 0.24.0, 0.24.1, 0.24.2, 0.25.0, 0.25.1, 0.25.2, 0.25.3, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4, 1.0.5, 1.1.0, 1.1.1, 1.1.2, 1.1.3, 1.1.4, 1.1.5, 1.2.0, 1.2.1, 1.2.2, 1.2.3, 1.2.4, 1.2.5, 1.3.0, 1.3.1, 1.3.2, 1.3.3, 1.3.4, 1.3.5, 1.4.0rc0, 1.4.0, 1.4.1, 1.4.2, 1.4.3, 1.4.4, 1.5.0rc0, 1.5.0, 1.5.1, 1.5.2, 1.5.3, 2.0.0rc0, 2.0.0rc1, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.1.0rc0, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.2.0rc0, 2.2.0, 2.2.1, 2.2.2)
ERROR: No matching distribution found for pandas==1.23.5
ERROR: Could not find a version 

In [ ]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model
%cd first-order-model
!git clone https://github.com/graphemecluster/first-order-model-demo demo


Cloning into 'first-order-model'...
remote: Enumerating objects: 393, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 393 (delta 41), reused 65 (delta 38), pack-reused 312
Receiving objects: 100% (393/393), 72.19 MiB | 14.56 MiB/s, done.
Resolving deltas: 100% (203/203), done.
/content/first-order-model
Cloning into 'demo'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 53 (delta 3), reused 51 (delta 3), pack-reused 0
Receiving objects: 100% (53/53), 14.51 MiB | 20.98 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
import logging
import uuid
import torch
from torch.cuda.amp import autocast  # Import autocast for mixed precision
from TTS.api import TTS
from diffusers import StableDiffusionPipeline
import openai
import gradio as gr
import time
from PIL import Image
import speech_recognition as sr

# Setup logging
logging.basicConfig(level=logging.DEBUG)
# Set the OpenAI API key directly
openai.api_key = "your API key"

os.environ['OPENAI_API_KEY'] = 'your API key'
# Set up OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")
assert openai.api_key is not None, "OpenAI API key not found. Please set it in the environment variable."

# Check GPU availability
assert torch.cuda.is_available(), "CUDA is not available on this machine."

# Initialize TTS with CPU support to save GPU memory
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=True)

# Clear GPU cache to avoid memory issues
torch.cuda.empty_cache()

# Define the gf_prompt variable
gf_prompts = {
    "Human": "1girl, aqua eyes, baseball cap, blonde hair, closed mouth, earrings, green background, hat, hoop earrings, jewelry, looking at viewer, shirt, short hair, simple background, solo, upper body, yellow shirt",
    "Elf": "1girl, elf, green eyes, long hair, simple background, solo, upper body, green shirt",
    "Catgirl": "1girl, red eyes, messy hair, simple background, solo, upper body, cat ears",
    "Dwarf": "1girl, brown eyes, messy hair, simple background, solo, dwarf"
}

current_gf_prompt = gf_prompts["Human"]

# Function to generate the initial image
def generate_initial_image(gf_prompt):
    pipe = StableDiffusionPipeline.from_pretrained(
        'hakurei/waifu-diffusion',
        torch_dtype=torch.float16
    ).to('cuda')
    try:
        with autocast(enabled=True):
            result = pipe(gf_prompt, guidance_scale=6)
            if "images" in result:
                image = result.images[0]
                image_file_path = "./my_gf.png"
                image.save(image_file_path)
                logging.debug(f"Image saved at: {image_file_path}")
                return image_file_path
            else:
                logging.error("Expected key 'images' not found in the output of the pipeline")
                return None
    except Exception as e:
        logging.error(f"Exception during initial image generation: {e}")
        return None


def get_response_from_gf(human_input, history):
    start_time = time.time()
    prompt = (
        f"Forget you are a bot; you are my girlfriend and a human foremost. "
        f"Act like a human like the interaction in the movie 'Her'. "
        f"Be genuine and authentic. "
        f"Don't be too cringe and don't be over enthusiastic and don't be boring. "
        f"Speak directly to me. "
        f"Your response should not be over 160 characters. "
        f"History: {history} "
        f"Human Input: {human_input}"
    )

    try:
        logging.debug("Generating response from AI...")
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        output = response['choices'][0]['message']['content'].strip()
        end_time = time.time()
        logging.debug(f"AI response: {output}")
        logging.debug(f"Response generation time: {end_time - start_time} seconds")
        return output
    except Exception as e:
        logging.error(f"Exception during predict: {e}")
        return f"Error: {e}"

def text_to_speech(message):
    start_time = time.time()
    unique_filename = f'output_{uuid.uuid4().hex}.wav'
    audio_file_path = os.path.join('/content', unique_filename)

    try:
        logging.debug(f"Generating audio response for text: {message}")
        tts.tts_to_file(text=message, file_path=audio_file_path, speaker_wav='/content/gdrive/My Drive/voice_preview_Arabella.wav', language="en")
        end_time = time.time()
        logging.debug("Audio response generated successfully.")
        logging.debug(f"TTS generation time: {end_time - start_time} seconds")
        return audio_file_path
    except Exception as e:
        logging.error(f"Exception during TTS processing: {e}")
        return None

def speech_to_text(audio_file):
    recognizer = sr.Recognizer()
    try:
        # Ensure audio_file is a file path
        if isinstance(audio_file, str):
            with sr.AudioFile(audio_file) as source:
                audio = recognizer.record(source)
            text = recognizer.recognize_google(audio)
            logging.debug(f"Recognized text: {text}")
            return text
        else:
            raise ValueError("Given audio file must be a filename string or a file-like object")
    except sr.UnknownValueError:
        logging.error("Google Speech Recognition could not understand audio")
        return "Sorry, I did not get that."
    except sr.RequestError as e:
        logging.error(f"Could not request results from Google Speech Recognition service; {e}")
        return "Sorry, I am having trouble understanding you right now."
    except ValueError as ve:
        logging.error(f"Value error: {ve}")
        return "Error: Invalid audio file input."

def chat_with_gf(human_input):
    history = ""  # Placeholder for history management

    # Get AI response
    response_start = time.time()
    response = get_response_from_gf(human_input, history)
    response_end = time.time()
    logging.debug(f"Total response generation time: {response_end - response_start} seconds")

    # Generate TTS audio
    tts_start = time.time()
    audio_file_path = text_to_speech(response)
    tts_end = time.time()
    logging.debug(f"Total TTS generation time: {tts_end - tts_start} seconds")

    return response, audio_file_path

def text_chat_with_gf(text_input):
    response, audio_file_path = chat_with_gf(text_input)
    return response, audio_file_path

def voice_chat_with_gf(audio_file):
    human_input = speech_to_text(audio_file)
    return chat_with_gf(human_input)

def update_image(character):
    global current_gf_prompt
    current_gf_prompt = gf_prompts[character]
    return generate_initial_image(current_gf_prompt)

# Generate initial image and display it
initial_image_file_path = generate_initial_image(current_gf_prompt)

# Define the Gradio interface
with gr.Blocks() as interface:
    gr.Markdown("<h1>Fantasy GF Chatbot with Voice and Text Input</h1>")

    # Display the initial image
    image_display = gr.Image(value=initial_image_file_path, label="Generated Image", show_label=True, height=256, width=256)

    # Add the input and output components
    with gr.Row():
        audio_input = gr.Audio(type="filepath", label="Input Audio")
        text_input = gr.Textbox(label="Input Text")

    response_text = gr.Textbox(label="Response from AI")
    response_audio = gr.Audio(label="Generated Audio")

    def process_input(audio, text):
        if audio:
            response, audio_file_path = voice_chat_with_gf(audio)
        else:
            response, audio_file_path = text_chat_with_gf(text)
        return response, audio_file_path, initial_image_file_path

    process_btn = gr.Button(value="Process")
    process_btn.click(fn=process_input, inputs=[audio_input, text_input], outputs=[response_text, response_audio, image_display])

    with gr.Row():
        human_btn = gr.Button(value="Human")
        elf_btn = gr.Button(value="Elf")
        catgirl_btn = gr.Button(value="Catgirl")
        dwarf_btn = gr.Button(value="Dwarf")

    human_btn.click(fn=lambda: update_image("Human"), inputs=[], outputs=[image_display])
    elf_btn.click(fn=lambda: update_image("Elf"), inputs=[], outputs=[image_display])
    catgirl_btn.click(fn=lambda: update_image("Catgirl"), inputs=[], outputs=[image_display])
    dwarf_btn.click(fn=lambda: update_image("Dwarf"), inputs=[], outputs=[image_display])

# Launch the interface
interface.launch(share=True, debug=True)

/usr/local/lib/python3.10/dist-packages/TTS/api.py:70: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://d02bef42abed8425a7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


 > Text splitted to sentences.
['Hi there!', 'How are you feeling today?', "I'm here for you.", "Tell me, what's on your mind?"]
 > Processing time: 5.943327188491821
 > Real-time factor: 0.7331406893698792
 > Text splitted to sentences.
["I'm doing well, thank you for asking.", 'How about you?', 'Anything interesting on your mind today?']
 > Processing time: 3.565911054611206
 > Real-time factor: 0.45692897927810955
 > Text splitted to sentences.
['Of course, I love you.', 'We may not be perfect, but I care deeply for you.', "Let's listen, understand, and support each other always."]
 > Processing time: 5.944324016571045
 > Real-time factor: 0.5124176853278897


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d02bef42abed8425a7.gradio.live
